In [21]:
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.axes._axes import Axes

from scipy.io import wavfile
from scipy.signal import stft,istft

In [4]:
#Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


<font color="red" size=6><b>Meta paramètres</b></font>

Comme calculer les spectrogrammes prend du temps, nous pouvons les calculer une fois pour toute puis les sauvegarder sur le disque. Toutefois, <font color="red"><b>ceci triple l'espace occupé sur disque</b></font>: passant de 6.3 Go à 17.2 Go. Veuillez donc préciser le paramètre `SAVE_SPECTROGRAMS` selon si vous pouvez utiliser cet espace.

In [5]:
DIRECTORY = Path("source_separation")
SAVE_SPECTROGRAMS = True

# Chargement des signaux, Visualisation et Dataset

## Chargement des signaux et spectrogrammes

In [6]:
train_folder = DIRECTORY / "train"
train_small_folder = DIRECTORY / "train_small"
test_folder = DIRECTORY / "test"
get_path = lambda folder,i : folder / ("000"+str(i))[-4:]

datasets_sizes = {
    train_small_folder : 50,
    train_folder : 5000,
    test_folder : 2000
}

Tous les signaux ont la même fréquence d'échantillonage, même longueur; et donc les mêmes f et t échantillonés pour le Spectrogramme.

In [7]:
def check_all_same(folder=train_small_folder):
    f_ref = None
    t_ref = None
    for i in range(datasets_sizes[train_small_folder]):
        fe,signal = wavfile.read(get_path(train_small_folder,i) / "voice.wav")
        len_signal = len(signal)
        f_spec,t_spec,spec = stft(
            signal,fs=fe,
            nperseg=400,nfft=512,noverlap=100)
        if f_ref is None: f_ref = f_spec ; t_ref = t_spec
        assert np.allclose(f_spec,f_ref)
        assert np.allclose(t_spec,t_ref)
    return fe,f_ref,t_ref

fe,f_ref,t_ref = check_all_same()
get_spectrogram = lambda signal : stft(signal,fs=fe,nperseg=400,nfft=512,noverlap=100)[2]

In [8]:
def load_signal_folder(folder: Path,
        load_signals=True,
        load_spectrograms=True) -> dict[str,dict]:
    """
    Return a dictionary with 3 sub dicts: "voice", "noise" and "mix"; and an "SNR" key.
    Each sub dict has 3 keys: "filename", "signal" and "spectrogram" 
    (except if load_signals or load_spectrograms are set to False)
    """
    keys = ["voice","noise","mix"]
    res = dict((k,dict()) for k in keys)
    for f in folder.iterdir():
        assert f.is_file()
        if "voice" in f.name: key = "voice"
        elif "noise" in f.name: key = "noise"
        else: 
            key = "mix"
            if f.suffix == ".wav":
                res["SNR"] = f.name.removesuffix(".wav").split("_")[-1]
        if f.suffix == ".wav" and load_signals:
            fe,signal = wavfile.read(f)
            res[key]["filename"] = f.name
            res[key]["signal"] = signal
        elif f.suffix == ".pt" and SAVE_SPECTROGRAMS and load_spectrograms: 
            # when SAVE_SPECTROGRAMS is False, we shouldn't be able to load them 
            # to save time, otherwise it's cheating.
            res[key]["spectrogram"] = torch.load(f,weights_only=True)
    # Create missing spectrograms
    if load_spectrograms:
        for key in keys:
            if "spectrogram" not in res[key]:
                assert load_signals
                spec = get_spectrogram(res[key]["signal"])
                res[key]["spectrogram"] = spec
                if SAVE_SPECTROGRAMS:
                    torch.save(torch.tensor(spec),folder / f"{key}_spectrogram.pt")
    return res


def remove_all_spectrograms():
    for folder in [train_folder,train_small_folder,test_folder]:
        for i in range(datasets_sizes[folder]):
            folder_i: Path = get_path(folder,i)
            for f in folder_i.iterdir():
                if "spectrogram" in f.name:
                    f.unlink()

Comme calculer les spectrogrammes prend du temps, nous pouvons les calculer une fois pour toute, en sauvegardant tous les spectrogrammes sur disque. Attention, <font color="red"><b>ceci triple l'espace occupé sur disque</b></font>: passant de 6.3 Go à 17.2 Go.
Pour les retirer utiliser: `remove_all_spectrograms()`.

In [9]:
def create_spectrograms(folder: Path):
    if SAVE_SPECTROGRAMS and not (folder/"0000"/"voice_spectrogram.pt").exists():
        for i in range(datasets_sizes[folder]):
            load_signal_folder(get_path(folder,i))

create_spectrograms(train_small_folder)
create_spectrograms(test_folder)
create_spectrograms(train_folder)

In [10]:
def compute_time(folder):
    time_load_signal = 0
    time_load_spec = 0
    time_spec = 0
    start = time.perf_counter()
    for i in range(datasets_sizes[folder]):
        fe,signal = wavfile.read(get_path(folder,i) / "voice.wav")
        time_load_signal += time.perf_counter() - start ; start = time.perf_counter()
        path_spec: Path = get_path(folder,i) / "voice_spectrogram.pt"
        if path_spec.exists():
            _ = torch.load(path_spec,weights_only=True)
        time_load_spec += time.perf_counter() - start ; start = time.perf_counter()
        _ = get_spectrogram(signal)
        time_spec += time.perf_counter() - start ; start = time.perf_counter()

    print(f"Temps total pour load {datasets_sizes[folder]} signaux: {time_load_signal}")
    print(f"Temps total pour en calculer les spectrogrammes: {time_spec}")
    print(f"Comparé au temps pour charger les spectrogrammes: {time_load_spec}")

print("Première fois:")
compute_time(test_folder)
print("\nDeuxième fois:")
compute_time(test_folder)

Première fois:
Temps total pour load 2000 signaux: 36.739902939647436
Temps total pour en calculer les spectrogrammes: 5.3411676697432995
Comparé au temps pour charger les spectrogrammes: 29.36331645771861

Deuxième fois:
Temps total pour load 2000 signaux: 1.0172409191727638
Temps total pour en calculer les spectrogrammes: 2.5520561523735523
Comparé au temps pour charger les spectrogrammes: 2.1508601531386375


On voit que charger les spectrogrammes pré-calculés est plus rapide que de les calculer à chaque fois. On note aussi une grosse différence entre la première fois qu'un fichier est chargé et la seconde. Nous pensons que le système place les derniers fichiers chargés dans le cache (recharger le notebook n'y change rien, donc la différence n'apparait que la toute première fois).

## Dataset

In [11]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,
            folder: Path,
            load_signals=True,
            load_spectrograms=False):
        self.folder = folder
        self.load_signals = load_signals
        self.load_spectrograms = load_spectrograms
            
    def __len__(self):
        return datasets_sizes[self.folder]
    
    def __getitem__(self,i):
        d = load_signal_folder(
            get_path(self.folder,i),
            load_signals=self.load_signals,
            load_spectrograms=self.load_spectrograms)
        ret = []
        for name in ["voice","noise","mix"]:
            if self.load_signals:
                ret.append(d[name]["signal"])
            if self.load_spectrograms:
                ret.append(d[name]["spectrogram"])
        ret.append(d["SNR"])
        return ret

Le Dataset peut contenir les signaux et/ou les spectrogrammes. De sorte à ne charger que le nécessaire. Exemple si on veut tout charger:

In [12]:
train_dataset = MyDataset(train_folder,load_signals=True,load_spectrograms=True)
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True)
print("Dataset length:",len(train_dataset))
for voice_signal,voice_spec,noise_signal,noise_spec,mix_signal,mix_spec,snr in train_dataloader:
    print("Signal's shape: ",voice_signal.shape)
    print("Spectrogram's shape: ",voice_spec.shape)
    print("SNRs :",snr)
    break

Dataset length: 5000


Signal's shape:  torch.Size([32, 80000])
Spectrogram's shape:  torch.Size([32, 257, 268])
SNRs : ('-1', '4', '0', '0', '-2', '2', '-2', '4', '0', '-2', '0', '2', '4', '3', '-2', '-4', '3', '3', '3', '-2', '-3', '0', '0', '2', '2', '3', '3', '3', '3', '1', '2', '0')


# Conv-tasnet: surpassing ideal time–frequency magnitude masking for speech separation.
(Inspired from Luo et al.)

In [13]:
train_dataset = MyDataset(
    train_folder,
    load_signals=False,
    load_spectrograms=True)
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True)

In [14]:
for voice_spec,noise_spec,mix_spec,snr in train_dataloader:
    print("Spec's shape:",voice_spec.shape)
    print("SNRs:",snr)
    break

Spec's shape: torch.Size([32, 257, 268])
SNRs: ('-2', '-1', '-2', '-1', '1', '1', '1', '4', '2', '4', '3', '-1', '3', '1', '4', '-2', '4', '2', '0', '1', '4', '-1', '3', '2', '0', '4', '0', '-3', '-4', '-1', '0', '-1')


In [15]:
class ConvTasNet(nn.Module):
    def __init__(self, N=256, L=20, B=256, H=512, P=3, X=8, R=4):
        super(ConvTasNet, self).__init__()
        self.encoder = nn.Conv1d(1, N, L, stride=L//2, bias=False)
        self.separator = nn.Sequential(
            nn.Conv1d(N, B, 1),
            nn.PReLU(),
            nn.Conv1d(B, B, 1),
            nn.PReLU(),
            nn.Conv1d(B, H, 1),
            nn.PReLU(),
            nn.Conv1d(H, H, P, groups=H, padding=P//2),
            nn.PReLU(),
            nn.Conv1d(H, B, 1),
            nn.PReLU(),
            nn.Conv1d(B, N, 1)
        )
        self.decoder = nn.ConvTranspose1d(N, 1, L, stride=L//2, bias=False)

    def forward(self, x):
        x = self.encoder(x)
        x = self.separator(x)
        x = self.decoder(x)
        return x

In [22]:
# Hyperparameters
N = 256
L = 20
B = 256
H = 512
P = 3
X = 8
R = 4
num_epochs = 10
learning_rate = 0.001

# Initialize model, loss function, and optimizer
model = ConvTasNet(N, L, B, H, P, X, R)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters())

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (voice_signal, noise_signal, mix_signal, snr) in enumerate(train_dataloader):
        mix_signal = mix_signal.unsqueeze(1)  # Add channel dimension
        optimizer.zero_grad()
        output = model(mix_signal)
        loss = criterion(output, voice_signal)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader)}")

print("Training complete.")

ModuleNotFoundError: No module named 'torch._dynamo'

In [18]:
class RefinedConvTasNet(nn.Module):
    def __init__(self, N=256, L=20, B=256, H=512, P=3, X=8, R=4):
        """
        Conv-TasNet implementation based on the paper.
        N: Number of filters in the encoder/decoder.
        L: Length of filters in samples.
        B: Number of channels in bottleneck and the residual paths.
        H: Number of channels in convolutional blocks.
        P: Kernel size in convolutional blocks.
        X: Number of convolutional blocks in each repeat.
        R: Number of repeats of the TCN.
        """
        super(RefinedConvTasNet, self).__init__()
        self.encoder = nn.Conv1d(1, N, kernel_size=L, stride=L // 2, bias=False)
        self.separator = TemporalConvNet(N, B, H, P, X, R)
        self.decoder = nn.ConvTranspose1d(N, 1, kernel_size=L, stride=L // 2, bias=False)

    def forward(self, x):
        encoded = self.encoder(x)  # [batch, N, time]
        masks = self.separator(encoded)  # [batch, N, time]
        separated = encoded * masks  # Apply masks
        output = self.decoder(separated)  # [batch, 1, time]
        return output

class TemporalConvNet(nn.Module):
    def __init__(self, N, B, H, P, X, R):
        """
        Temporal Convolutional Network (TCN) for separator.
        """
        super(TemporalConvNet, self).__init__()
        layers = []
        for r in range(R):
            for x in range(X):
                dilation = 2 ** x  # Exponential dilation
                layers.append(ConvBlock(N, B, H, P, dilation))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class ConvBlock(nn.Module):
    def __init__(self, N, B, H, P, dilation):
        """
        Single convolutional block in the TCN.
        """
        super(ConvBlock, self).__init__()
        self.layer_norm = nn.LayerNorm([N, 1])
        self.depthwise_conv = nn.Conv1d(B, B, kernel_size=P, dilation=dilation, padding=dilation, groups=B)
        self.pointwise_conv = nn.Conv1d(B, N, kernel_size=1)
        self.activation = nn.PReLU()

    def forward(self, x):
        x = self.layer_norm(x.transpose(1, 2)).transpose(1, 2)  # Layer normalization
        x = self.depthwise_conv(x)
        x = self.pointwise_conv(x)
        return self.activation(x)

# Hyperparameters for testing
N, L, B, H, P, X, R = 256, 20, 256, 512, 3, 8, 4
sample_model = RefinedConvTasNet(N, L, B, H, P, X, R)

# Sample input to verify architecture
sample_input = torch.randn(4, 1, 32000)  # [batch, channels, time]
sample_output = sample_model(sample_input)

# Confirm output shape matches input shape
sample_input.shape, sample_output.shape


RuntimeError: Given normalized_shape=[256, 1], expected input with shape [*, 256, 1], but got input of size[4, 3199, 256]